In [1]:
!pip3 install nltk
!pip3 install transformers
!pip3 install torch torchvision torchaudio


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import json
import pandas as pd
import nltk
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

## Testing

In [3]:
itemid, shopid = "19043502047", "1358899"
limit = 50

In [86]:
url = "https://shopee.sg/api/v2/item/get_ratings"

querystring = {
    "exclude_filter":"1",
    "filter":"1", ##! change 1 for those with comments!!!!!!!!
    "filter_size":"0",
    "flag":"1",
    "fold_filter":"0",
    "itemid":itemid,
    "limit":str(limit),
    "offset":"2124",
    "relevant_reviews":"false",
    "request_source":"2",
    "shopid":shopid,
    "tag_filter":"",
    "type":"0",
    "variation_filters":""}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0"
}

response = requests.request("GET", url, headers=headers, params=querystring)

response

<Response [200]>

In [87]:
response.json()['data']['item_rating_summary']

{'rating_total': 4339,
 'rating_count': [20, 18, 85, 328, 3888],
 'rcount_with_context': 2083,
 'rcount_with_image': 1702,
 'rcount_with_media': 1714,
 'rcount_local_review': 4339,
 'rcount_repeat_purchase': 0,
 'rcount_overall_fit_small': 0,
 'rcount_overall_fit_fit': 0,
 'rcount_overall_fit_large': 0,
 'rcount_oversea_review': 0,
 'rcount_folded': 0}

In [89]:
response.json()['data']

{'ratings': None,
 'item_rating_summary': {'rating_total': 4339,
  'rating_count': [20, 18, 85, 328, 3888],
  'rcount_with_context': 2083,
  'rcount_with_image': 1702,
  'rcount_with_media': 1714,
  'rcount_local_review': 4339,
  'rcount_repeat_purchase': 0,
  'rcount_overall_fit_small': 0,
  'rcount_overall_fit_fit': 0,
  'rcount_overall_fit_large': 0,
  'rcount_oversea_review': 0,
  'rcount_folded': 0},
 'is_sip_item': False,
 'rcmd_algo': 'BUNDLE:comment_search,RECALLER:comment_search_default,QUEUE:comment_search_default,NEWABTEST:0,QUEUES:comment_search_default|comment_search_default',
 'downgrade_switch': False,
 'has_more': False,
 'show_local_review': False,
 'browsing_ui': '',
 'enable_buyer_gallery_media': True,
 'user_latest_rating': None,
 'size_info_abt': '',
 'top_ratings': [],
 'resize_image_abt': False,
 'purchase_bar_abt': 'bucket_a',
 'tag_filters': [],
 'signature': ''}

In [7]:
test_df = pd.json_normalize(response.json()['data']['ratings'])
test_df.columns

Index(['orderid', 'itemid', 'cmtid', 'ctime', 'rating', 'userid', 'shopid',
       'comment', 'rating_star', 'status', 'mtime', 'editable', 'opt',
       'filter', 'mentioned', 'is_hidden', 'can_follow_up', 'follow_up',
       'submit_time', 'author_username', 'author_portrait', 'author_shopid',
       'anonymous', 'images', 'videos', 'product_items', 'delete_reason',
       'delete_operator', 'ItemRatingReply', 'tags', 'editable_date',
       'show_reply', 'like_count', 'liked', 'sync_to_social',
       'exclude_scoring_due_low_logistic', 'loyalty_info', 'template_tags',
       'has_template_tag', 'sync_to_social_toggle', 'is_repeated_purchase',
       'display_variation_filter', 'overall_fit', 'is_normal_item', 'viewed',
       'show_view', 'sync_to_social_detail', 'profile', 'size_info_tags',
       'size_info_abt', 'image_data', 'is_super_review', 'super_reviewer_tag',
       'is_newly_created', 'template_hints', 'template_tags_hints', 'region',
       'template_abt', 'is_repeat_ed

## Scraping

In [90]:
def get_reviews_shopee(itemid, shopid, limit=None, limit_per_req = 59, offset=0): 
    url = "https://shopee.sg/api/v2/item/get_ratings"

    querystring = {
        "exclude_filter":"1",
        "filter":"1", #! 1 only includes those with comments
                      #! 0 includes all with/without comments
        "filter_size":"0",
        "flag":"1",
        "fold_filter":"0",
        "itemid":itemid,
        "limit":str(limit_per_req),
        "offset":str(offset),
        "relevant_reviews":"false",
        "request_source":"2",
        "shopid":shopid,
        "tag_filter":"",
        "type":"0",
        "variation_filters":""}

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    rating_total = response.json()['data']['item_rating_summary']['rating_total']
    rating_count = response.json()['data']['item_rating_summary']['rating_count']
    rcount_with_context = response.json()['data']['item_rating_summary']['rcount_with_context']

    # return response.json()['data']['ratings']
    # return rating_count, rcount_with_context

    ratings_list = list()
    if limit is None:
        limit = rcount_with_context

    pages_to_scrape = limit//limit_per_req + 1
    for i in range(pages_to_scrape):
        querystring['offset'] = str(offset)
        response = requests.request("GET", url, headers=headers, params=querystring)
        # print(offset)
        # if 'ratings' in response.json()['data']:
        ratings_list += response.json()['data']['ratings']
        # print(response.json()['data']['ratings'])
        offset += limit_per_req
    
    ratings_df = pd.json_normalize(ratings_list)
    # ratings_df.columns
    return ratings_df

In [91]:
itemid, shopid = "19043502047", "1358899"
df = get_reviews_shopee(itemid, 
                        shopid, 
                        # limit =100,
                        )
df.head()

,orderid,itemid,cmtid,ctime,rating,userid,shopid,comment,rating_star,status,...,ItemRatingReply.shopid,ItemRatingReply.comment,ItemRatingReply.rating_star,ItemRatingReply.status,ItemRatingReply.mtime,ItemRatingReply.editable,ItemRatingReply.opt,ItemRatingReply.filter,ItemRatingReply.mentioned,ItemRatingReply.is_hidden
0,136948335232700,19043502047,11430713796,1683378046,1,225829130,1358899,Quality:very good\n\nFast delivery. Next day! ...,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,129044608299967,19043502047,10696112443,1675480373,1,444946297,1358899,Quality:honestly not bad 😋😋\n\nbought 2 during...,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,131733619237247,19043502047,10966574367,1678432244,1,129101896,1358899,"Fast delivery, around 2 days. Packaging is goo...",5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,138643853216090,19043502047,11585703328,1685069662,1,127192298,1358899,Received very quickly. Great quality and super...,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,158038783274918,19043502047,13842720932,1704954931,1,279266001,1358899,It took 3 days to arrive with good condition. ...,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
print(df.shape)

(2083, 91)


In [93]:
df['comment'].value_counts()

Quality:good                                                                        13
Fast delivery                                                                        5
Good                                                                                 3
Quality:good\n\ngood                                                                 3
Good quality                                                                         3
                                                                                    ..
Quality:Good\n\nThe colour is so pretty. Looks really good. Happy with purchase.     1
Fast delivery. Good quality. Feels like a size bigger. But nice                      1
Item received with good condition, fast delivery. But too small …                    1
Quality:good\n\nDelivery is fast. The slipper fits perfectly and is so cute!         1
good👍🏻                                                                               1
Name: comment, Length: 2040, dtype: int64

In [96]:
df[df['rating_star'] <3]

,orderid,itemid,cmtid,ctime,rating,userid,shopid,comment,rating_star,status,...,ItemRatingReply.shopid,ItemRatingReply.comment,ItemRatingReply.rating_star,ItemRatingReply.status,ItemRatingReply.mtime,ItemRatingReply.editable,ItemRatingReply.opt,ItemRatingReply.filter,ItemRatingReply.mentioned,ItemRatingReply.is_hidden
262,143705603294962,19043502047,12102988478,1690466037,0,7225494,1358899,Received with good condition. But size run sma...,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415,116896532260836,19043502047,9598486296,1663518442,-1,295988533,1358899,lots of defects. tried to contact seller but i...,1,2,...,NaN,"Dear buyer, we are sorry for your experience. ...",NaN,NaN,1.663638e+09,NaN,NaN,NaN,NaN,False
693,137893252247352,19043502047,11605905386,1685286448,0,15172543,1358899,Quality:decent\n\nnah the sizes just ain’t it ...,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,131429283259533,19043502047,10913575420,1677923607,0,153382131,1358899,Quality:overall can see minor quality defects;...,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
981,141980705236581,19043502047,11913445722,1688623094,0,159794894,1358899,"Quality:styrofoamy, very light\n\nSent out lat...",2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1040,124978738239001,19043502047,10392152799,1671852591,0,22740961,1358899,Quality:ok 7/10\n\nDelivery was fast. Need to ...,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1045,131551300295465,19043502047,10934737494,1678120364,-1,134053990,1358899,Quality:quality is gd but one of the grey one ...,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1059,131983120214119,19043502047,10975968453,1678526619,-1,563646162,1358899,Quality:sucks booty\n\nmaterial is hard and u ...,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1095,152642831318917,19043502047,13232032838,1700133767,-1,696876926,1358899,I thought it would be good but difficult to ru...,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1096,130753126298652,19043502047,10857490968,1677291327,-1,118714741,1358899,Quality:seems bad\n\nafter one wear the grip i...,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sentiment Analysis

In [21]:
ROBERTA_MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(ROBERTA_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(ROBERTA_MODEL)

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

c:\Users\Claudeon\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Claudeon\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [101]:
BERT_MODEL = f"nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL)

In [100]:
def roberta_classification(text, debug=False):
    # Run for Roberta Model
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    if debug:
        print(text)
    return scores_dict
roberta_classification(df['comment'][262], debug=True)

Received with good condition. But size run small. My previous pairs size 42-43 and now size 44-45 is almost the same size. And is not comfortable to wear .  look at the slippers, the new slippers the front is so high up. It does not feel secure when walk. Not nice, some more I bought 2 pairs. Sigh….


{'roberta_neg': 0.85082173,
 'roberta_neu': 0.12977569,
 'roberta_pos': 0.019402608}

In [102]:
def bert_classification(text, debug=False):
    # Run for BERT Model
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    if debug:
        print(text)
    return scores
bert_classification(df['comment'][262], debug=True)

Received with good condition. But size run small. My previous pairs size 42-43 and now size 44-45 is almost the same size. And is not comfortable to wear .  look at the slippers, the new slippers the front is so high up. It does not feel secure when walk. Not nice, some more I bought 2 pairs. Sigh….


array([-0.00566801,  2.0982053 ,  2.1872063 , -0.11072581, -3.3476822 ],
      dtype=float32)